# https://github.com/kernoelpanic/smartenv

# Smart Contract Crash Course - Basic interaction with `ganache-cli`

The most common client lib to interface with ethereum clients (`geth`,`ganache`,...) is **web3**.
* [web3.js](https://github.com/ethereum/web3.js/)
    - node.js version
    - [API](https://web3js.readthedocs.io/)
* [web3py](https://github.com/ethereum/web3.py)
    - python version
    - as I prefer python, we will use this one ;)
    - [API](https://web3py.readthedocs.io/)

For the tutorial we will only interact with a local EVM compatible Ethereum emulation softare, which uses his own local development testnet blockchain. 
* ~~[ganache-cli](https://github.com/trufflesuite/ganache-cli/tree/master)~~ Depricated by `ganache`
    - [API](https://github.com/trufflesuite/ganache-cli#implemented-methods)
* ~~[ganache](https://github.com/trufflesuite/ganache/tree/master)~~ Depricated 
    - [ganache cli options](https://trufflesuite.com/docs/ganache/reference/cli-options/)
* [anvil](https://github.com/foundry-rs/foundry/tree/master/crates/anvil)

## Connect via RPC / HTTP
Import the web3 libarary and connect to a running `ganache-cli` node via `HTTPProvider`
For the tutorial configuration the docker container of ganache is located at `172.18.0.2`.

In [1]:
import web3

w3 = web3.Web3(web3.Web3.HTTPProvider("http://172.18.0.2:8545"))
# check if connection was successful
assert w3.is_connected()

In [2]:
# Client version, should be TestRPC
w3.client_version

'anvil/v0.2.0'

In [3]:
# display network ID of client you are connected to, should be a random int
w3.net.version

'31337'

In [4]:
# number of connected peers to our peer should be 0
# not available in anvil (foundry)
#w3.net.peer_count

## Blocks

In [5]:
# return current blockchain head of node
# Since ganache "simulates" blokchain,
# blocks are mined on demand.
# Therefore, no block has been mined yet
w3.eth.block_number

0

In [6]:
# block 0 ist the genesis block
genesis_block = w3.eth.get_block(0)
genesis_block

AttributeDict({'hash': HexBytes('0x4fb2bcc3dd200071734c9ebd9916d1c4fb7b4be93b424d9d80300aea0c5420ed'),
 'parentHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'miner': '0x0000000000000000000000000000000000000000',
 'stateRoot': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRoot': HexBytes('0x56e81f171bcc55a6ff8345e692c0f86e5b48e01b996cadc001622fb5e363b421'),
 'receiptsRoot': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'number': 0,
 'gasUsed': 0,
 'gasLimit': 80000000,
 'extraData': HexBytes('0x'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

As you see, the genesis block is special as it is the only block with no parent hash:

In [7]:
genesis_block['number']

0

In [8]:
genesis_block['parentHash']

HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000')

In [9]:
# instruct our test blockchain to mine a block
w3.provider.make_request('evm_mine',params='')

{'jsonrpc': '2.0', 'id': 5, 'result': '0x0'}

In [10]:
w3.eth.block_number

1

In [11]:
first_block = w3.eth.get_block(1)
first_block

AttributeDict({'hash': HexBytes('0x588728c6e5359320956b557160c9abca01d63bc8428cd4292142f976fbdb0930'),
 'parentHash': HexBytes('0x4fb2bcc3dd200071734c9ebd9916d1c4fb7b4be93b424d9d80300aea0c5420ed'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'miner': '0x0000000000000000000000000000000000000000',
 'stateRoot': HexBytes('0x8a69ea7b553ac5901695f9b641dc9585ba994d6aa5256ea419c584eac2ac9456'),
 'transactionsRoot': HexBytes('0x56e81f171bcc55a6ff8345e692c0f86e5b48e01b996cadc001622fb5e363b421'),
 'receiptsRoot': HexBytes('0x56e81f171bcc55a6ff8345e692c0f86e5b48e01b996cadc001622fb5e363b421'),
 'number': 1,
 'gasUsed': 0,
 'gasLimit': 80000000,
 'extraData': HexBytes('0x'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [12]:
first_block['number']

1

In [13]:
first_block['parentHash']

HexBytes('0x4fb2bcc3dd200071734c9ebd9916d1c4fb7b4be93b424d9d80300aea0c5420ed')

In [14]:
# our node is configured to mine blocks 
# not available in anvil (foundry)
#w3.eth.mining

In [15]:
# but since it is a fake chain we have no hashrate
# not available in anvil (foundry)
#w3.eth.hashrate

## Externally Owned Accounts

Accounts are your **external accounts** consisting of public and private keys. 
The ethereum address are the last 40 hex characters (20 bytes) of the hash (Keccak-256) of the public key prefixed with `0x`

`ganache` generates a list of accounts on startup. 
For the challenges you will recieve your personal account keypair which you have to copy in your `geth` datadir folder to import it. 

In [16]:
w3.eth.accounts

['0xf04e3ce2eb267e94463370BaA06ef1F2f8487298',
 '0xd4e5A8028D2d695a9b2262DE611d9B5531C3a2F1',
 '0xF3E56417F52DF9D4b901381ADdf42D476237652f',
 '0xAba2E560dD60122cCd8DEf66FF8bc2B811ca80c7',
 '0xC36B045DF31C3fCA4F6F82Ec564F63aECe37BE52',
 '0x9BafaA98A14592972F999ed27d31643Fe36d8774',
 '0xbb9335Be8E32cB1A8E28DeA0AdcAd6ad38CF409e',
 '0x00C318c85a1880d651e219B91E1AadA2b391af9e',
 '0x8935Bd73abD8F979e248c103542E5b50C419e20a',
 '0x2a157DF71dF10B376345b938A2C779a0d694190C']

In [17]:
len(w3.eth.accounts)

10

In [18]:
print("address length with 0x at the beginning = ",len(w3.eth.accounts[0]))
w3.eth.accounts[0]

address length with 0x at the beginning =  42


'0xf04e3ce2eb267e94463370BaA06ef1F2f8487298'

**Note:** Ethereum uses Keccak-256 but often refers to it as SHA3, but the output of SHA3 as specified by NIST is different!

In [19]:
from sha3 import sha3_256
sha3_256(b'').hexdigest()

'a7ffc6f8bf1ed76651c14756a061d662f580ff4de43b49fa82d80a4b80f8434a'

In [20]:
from sha3 import keccak_256
keccak_256(b'').hexdigest()

'c5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470'

In [21]:
w3.keccak(text="").hex()[2:] # since version 6 sha3 has been renamed to keccak!

'c5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470'

In [22]:
assert keccak_256(b'').hexdigest() == w3.keccak(text="").hex()[2:]

The default address (also used to receive rewards from mining blocks) is usually the first in the list of accounts and usually also accessible via `w3.eth.coinbase` but this changed. Moreover coinbase is no longer available in `anvil`

In [23]:
# not available in anvil (foundry)
#w3.eth.coinbase

In [24]:
# w3.eth.coinbase = w3.eth.accounts[0]

It is good practise to specifically assing a `default_account` some functions might cause problems if this is not explicitly assigned.

In [25]:
w3.eth.default_account = w3.eth.accounts[0]
w3.eth.default_account 

'0xf04e3ce2eb267e94463370BaA06ef1F2f8487298'

There is also a helper method to check if a string is an ethereum address.

In [26]:
w3.is_address(w3.eth.accounts[0])

True

In [27]:
test_addr = '0xd3cda913deb6f67967b99d67acdfa1712c293601'
print("address\t\t\t: {}\nlength\t\t\t: {}\nw3.is_address()\t\t: {}".format(test_addr,
                                              len(test_addr),
                                              w3.is_address(test_addr)))
print()
print("\nWrong address:")
print("address\t\t\t: {}\nlength\t\t\t: {}\nw3.is_address()\t\t: {}".format(test_addr[:-1],
                                              len(test_addr[:-1]),
                                              w3.is_address(test_addr[:-1])))

address			: 0xd3cda913deb6f67967b99d67acdfa1712c293601
length			: 42
w3.is_address()		: True


Wrong address:
address			: 0xd3cda913deb6f67967b99d67acdfa1712c29360
length			: 41
w3.is_address()		: False


**Note:** The capital letters of an address encode an optional checksum
https://github.com/ethereum/EIPs/blob/master/EIPS/eip-55.md

In [28]:
print("address\t\t\t: {}\nlength\t\t\t: {}\nw3.is_address()\t\t: {}\nw3.is_checksum_address(): {}".format(test_addr,
                                              len(test_addr),
                                              w3.is_address(test_addr),
                                              w3.is_checksum_address(test_addr)))

address			: 0xd3cda913deb6f67967b99d67acdfa1712c293601
length			: 42
w3.is_address()		: True
w3.is_checksum_address(): False


In [29]:
test_addr = w3.to_checksum_address(test_addr); test_addr # convert addr to checkusm addr

'0xd3CdA913deB6f67967B99D67aCDFa1712C293601'

In [30]:
print("address\t\t\t: {}\nlength\t\t\t: {}\nw3.is_address()\t\t: {}\nw3.is_checksum_address(): {}".format(test_addr,
                                              len(test_addr),
                                              w3.is_address(test_addr),
                                              w3.is_checksum_address(test_addr)))

address			: 0xd3CdA913deB6f67967B99D67aCDFa1712C293601
length			: 42
w3.is_address()		: True
w3.is_checksum_address(): True


### Example: Create externally owned account 

To see what an externally controlled account actually is, lets create one. 
First useing `geth` then with `web3.py`.

In [31]:
!pwd 

/smartenv/course


In [32]:
TEST_PATH = "/smartenv/gitignore/testaccount/"

In [33]:
!echo {TEST_PATH} # or !echo $TEST_PATH

/smartenv/gitignore/testaccount/


In [34]:
!mkdir -p {TEST_PATH}

In [35]:
!echo "password" > {TEST_PATH}pwd.txt

In [36]:
!geth --maxpeers 0 --datadir={TEST_PATH} --password={TEST_PATH}/pwd.txt account new

INFO [12-27|20:31:47.304] Maximum peer count                       ETH=0 LES=0 total=0
INFO [12-27|20:31:47.306] Smartcard socket not found, disabling    err="stat /run/pcscd/pcscd.comm: no such file or directory"

Your new key was generated

Public address of the key:   0x81Aec9179d3DC6aB4e352811A4aE8b03f132e2B0
Path of the secret key file: /smartenv/gitignore/testaccount/keystore/UTC--2023-12-27T20-31-47.307310899Z--81aec9179d3dc6ab4e352811a4ae8b03f132e2b0

- You can share your public address with anyone. Others need it to interact with you.
- You must NEVER share the secret key with anyone! The key controls access to your funds!
- You must BACKUP your key file! Without the key, it's impossible to access account funds!
- You must REMEMBER your password! Without the password, it's impossible to decrypt the key!



In [37]:
keystore_file = !ls {TEST_PATH}/keystore
keystore_file[0]

'UTC--2023-12-12T10-31-04.271291333Z--ebd9a7d2598068499d5885acf21d3a46248497e0'

In [38]:
with open(TEST_PATH + 'keystore/' + keystore_file[0]) as keyfile:
    keyfile_json = keyfile.read()
import json
json.loads(keyfile_json)

{'address': 'ebd9a7d2598068499d5885acf21d3a46248497e0',
 'crypto': {'cipher': 'aes-128-ctr',
  'ciphertext': 'cc7dcd7bf46885e1f98f8a3b4c7a8a0ea9320a96a079397a2ef3cf6c363ff546',
  'cipherparams': {'iv': '82b49f0542add4583e6a07d53e43e3c7'},
  'kdf': 'scrypt',
  'kdfparams': {'dklen': 32,
   'n': 262144,
   'p': 1,
   'r': 8,
   'salt': '1f94c220b439994efbfef9a02eed44fe5d4ad89aba3b1a0eabe31e8174a9eabc'},
  'mac': '0e3261a403277395b0d0c4466d53220dc6cbc2b3e0f5f83b5d710e77321dc44c'},
 'id': 'dba0353b-3367-4999-ad70-48a64a4e5a5c',
 'version': 3}

In [39]:
private_key = w3.eth.account.decrypt(keyfile_json,"password")

In [40]:
private_key.hex()

'0x63a4765d0899975947331c24da9d0991626edf15c2f3f3ffafc7d57936ea5f0a'

In [41]:
#account = w3.eth.account.privateKeyToAccount(private_key) # depricated
account = w3.eth.account.from_key(private_key)

In [42]:
account.address # this is the same as in the geth output above and the json file

'0xebD9A7D2598068499d5885ACf21D3a46248497e0'

In [43]:
#account.private_key.hex()
account.key.hex() # get the private key

'0x63a4765d0899975947331c24da9d0991626edf15c2f3f3ffafc7d57936ea5f0a'

In [44]:
account.encrypt("password") # create a keyfile again, this time salt is different

{'address': 'ebD9A7D2598068499d5885ACf21D3a46248497e0',
 'crypto': {'cipher': 'aes-128-ctr',
  'cipherparams': {'iv': '33f28103116a2b142f4ae995157ef1c6'},
  'ciphertext': 'a26110a45c2e62522231e5687448b45796681288873701c8c3c3d60068fcc39d',
  'kdf': 'scrypt',
  'kdfparams': {'dklen': 32,
   'n': 262144,
   'r': 8,
   'p': 1,
   'salt': '2e393fbf6f681de83cea6506acd94dc4'},
  'mac': '8b9fbf9aaf30bfd73e0681722a35e4a295c0c4fa5842665e4c9954301564aa08'},
 'id': 'd1ec27ba-92c9-44d6-b8eb-4875e833670d',
 'version': 3}

Lets create an account (new private/public key pair) directly with web3.py:

In [45]:
account = w3.eth.account.create("random string to increase entropy")

In [46]:
account.address

'0xD2FBdf8c364c415bf0be2e04aDBef30aC2ae900f'

In [47]:
account.key.hex()

'0x8dee944ccd6cc91add64e041270b3648044e7f1e180a7064ba71a2c37dec00b1'

In [48]:
account.encrypt("password")

{'address': 'D2FBdf8c364c415bf0be2e04aDBef30aC2ae900f',
 'crypto': {'cipher': 'aes-128-ctr',
  'cipherparams': {'iv': 'f2bde98a275d03294174ea9b61f2524e'},
  'ciphertext': '49e70fa77672b355a28474157536e0319592a879542e316fe614c4f063b8050c',
  'kdf': 'scrypt',
  'kdfparams': {'dklen': 32,
   'n': 262144,
   'r': 8,
   'p': 1,
   'salt': '14155b79f89fe76e3c6781edfa1bf82a'},
  'mac': '62acee90c4e16e3f554d4b7f9a8ce6d4d885090f1796e23ae4716de24983e538'},
 'id': 'e598a45f-ae20-4cab-be16-205a71ec8d31',
 'version': 3}

In [49]:
# generate a static account for the miner for testing purposes
mineraccount = w3.eth.account.from_key(bytes.fromhex('63a4765d0899975947331c24da9d0991626edf15c2f3f3ffafc7d57936ea5f0a'))

In [50]:
mineraccount.address

'0xebD9A7D2598068499d5885ACf21D3a46248497e0'

## Balance
To query the balance of an account or contract the function `eth.getBalance()` can be used with the respective address.

**Note:** Per default all balances are in *Wei* i.e., the smallest unit of value in Ethereum. 
$$
    1 \textrm{ ether} = 10^{18} \textrm{ wei} 
$$

In [51]:
balance = w3.eth.get_balance(w3.eth.accounts[0])
balance # in wei

1000000000000000000000

In [52]:
balance * 10**-18 # in ether

1000.0000000000001

In [53]:
# Directly output the balance in ether from wei
w3.from_wei(balance,'ether')

Decimal('1000')

In [54]:
# Directly output the balance in wei from ether
w3.to_wei(100,'ether')

100000000000000000000

In [55]:
def get_balance(address,unit="ether"):
    if unit.lower() == "ether":        
        return w3.from_wei(w3.eth.get_balance(address),'ether')
    elif unit.lower() == "wei":
        return w3.eth.get_balance(address)
    elif unit.lower() == "gwei":
        return w3.eth.get_balance(address) / 10**9
    else:
        assert False,"Invalid unit given"

In [56]:
get_balance(w3.eth.accounts[0],"ether") 

Decimal('1000')

In [57]:
get_balance(w3.eth.accounts[0],"wei") 

1000000000000000000000

In [58]:
get_balance(w3.eth.accounts[0],"gwei") 

1000000000000.0

In [59]:
# each of the ganache test account has initially 100 ether balance
for account in w3.eth.accounts:
    print(get_balance(account))

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


In [60]:
import requests 

def get_exchange_rate(xfrom="ETH",xto="EUR"):
    """ Query current exchange rate """
    api_endpoint = f"https://min-api.cryptocompare.com/data/price?fsym={xfrom}&tsyms={xto}"
    response = requests.get(api_endpoint)
    if response.status_code == 200:
        json_data = response.json()
        return json_data[xto]
    else:
        print(f"Error: {response.status_code}")
        return None

In [61]:
ETH_X_EUR = get_exchange_rate(); ETH_X_EUR

2115.43

In [62]:
import requests 
def get_current_proposed_baseFeePerGas(unit="gwei"):
    """ Query current suggested baseFeePerGas """
    api_endpoint = f"https://api.etherscan.io/api?module=gastracker&action=gasoracle&apikey=YourApiKeyToken"
    response = requests.get(api_endpoint)
    if response.status_code == 200:
        json_data = response.json()
        #print(json_data)
        base_fee_gwei = json_data["result"]["suggestBaseFee"]
        if unit.lower() == "gwei":
            return float(base_fee_gwei)
        elif unit.lower() == "wei":
            return int(float(base_fee_gwei) * 10**9)
        else:
            assert False,"Unimplemented unit type"
    else:
        print(f"Error: {response.status_code}")
        return None

In [63]:
ETH_BASE_FEE_PER_GAS_WEI = get_current_proposed_baseFeePerGas("wei"); ETH_BASE_FEE_PER_GAS_WEI

32068701964

## Payment Transaction and Gas

We now send a transaction that just transfers value i.e., ether from one account to another. Thereby, `ganache` automatically mines a block once it sees that a transactions requires confirmation. 

**Note:** When connected to `geth` or `openethereum` you have to unlock an account first to send a transaction since the private keys are stored encrypted per default and protected with a password. In `ganache` this is not necessary since it is a development environment.

In [64]:
# required in geth but not in ganache
# Parameters are:
# * account
# * password
# * duration of unlock (if 0 then forever)
# (not required in anvil)
#w3.geth.personal.unlock_account(w3.eth.accounts[0],"",0)

In [65]:
w3.eth.block_number

1

In [66]:
w3.eth.send_transaction({'from':w3.eth.accounts[0], 
                        'to':w3.eth.accounts[1], 
                        'value':10**18})

HexBytes('0x93ff785f6eb2fe9ff1ed12ac868d286fc0ea0070f1506c94ef51e797bee274bf')

In [67]:
blkNumber = w3.eth.block_number
blkNumber

2

In [68]:
# the balance is reduced by more than 1 ether
# Why do you think this is the case?
get_balance(w3.eth.accounts[0])

Decimal('998.999981625')

In [69]:
get_balance(w3.eth.accounts[1]) # the balance at the destination is increased by 1 ether

Decimal('1001')

In [70]:
difference_wei = w3.to_wei(999,"ether") - get_balance(w3.eth.accounts[0],"wei"); difference_wei

18375000000000

In [71]:
assert difference_wei + get_balance(w3.eth.accounts[0],"wei") == w3.to_wei(999,"ether")

Since EIP 1559 gase prices are handled differently: https://ethereum.org/en/developers/docs/gas/

* **Base fee**
  + mimimum fee that has to be paid by every transaction (higher during high load, lower during low load)
  + set by the network itself, not by miners. The base fee changes block by block, based on how full the previous block was.
  + max inc./dec. 12.5 % per block
  + the entire base fee is entirely burned! This does not go to the miner!
* **Priority fee** (miner **tip**)
  + Additional fee paid to the miner to indicate "priority"

**!!Attention!!** The implementation of gas mechanics might differ depending
on the underlying EVM/consensus engine implementation (e.g., ganache, anvil, geth, ...).
Roughly, it should work like this:

If `maxPriorityFeePerGas > 0` (e.g., ganache): 
* The fee you pay in wei is calculated by `maxFeePerGas * gasUsed` 
* You always pay `baseFeePerGas`, if that is below `maxFeePerGas` the rest will be paid to the miner as tip if `maxPriorityFeePerGas` is above zero.
    + `maxFeePerGas - baseFeePerGas = tip`
 
If `maxPriorityFeePerGas == 0` (e.g., anvil):
* Only the `baseFeePerGas` value is used. No tip is given. 
* The total fee you pay in wei is calculated `baseFeePerGas * gasUsed`

In [72]:
# get first (and only) transaction in the previously mined block
tx = w3.eth.get_transaction_by_block(blkNumber,0)
tx 

AttributeDict({'hash': HexBytes('0x93ff785f6eb2fe9ff1ed12ac868d286fc0ea0070f1506c94ef51e797bee274bf'),
 'nonce': 0,
 'blockHash': HexBytes('0x076df6ea9e7f09ec7dd186f54c58188299535d5c9a19b66208fb25314775fe33'),
 'blockNumber': 2,
 'transactionIndex': 0,
 'from': '0xf04e3ce2eb267e94463370BaA06ef1F2f8487298',
 'to': '0xd4e5A8028D2d695a9b2262DE611d9B5531C3a2F1',
 'value': 1000000000000000000,
 'gasPrice': 875000000,
 'gas': 121000,
 'input': HexBytes('0x'),
 'v': 1,
 'r': HexBytes('0x54088000a67330b605515b0a422abcf82407f4d80ecf7b94d61501e59af42ccf'),
 's': HexBytes('0x33281a5adc68996460423236b7c5970b80deba5bac0c7a9be01d49b88f8ba66d'),
 'sourceHash': '0x0000000000000000000000000000000000000000000000000000000000000000',
 'isSystemTx': False,
 'type': 2,
 'accessList': [],
 'maxPriorityFeePerGas': 0,
 'maxFeePerGas': 1875000000,
 'chainId': 31337})

In [73]:
# get block and see the transaction id and gas used below
blk = w3.eth.get_block(blkNumber)
blk

AttributeDict({'hash': HexBytes('0x076df6ea9e7f09ec7dd186f54c58188299535d5c9a19b66208fb25314775fe33'),
 'parentHash': HexBytes('0x588728c6e5359320956b557160c9abca01d63bc8428cd4292142f976fbdb0930'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'miner': '0x0000000000000000000000000000000000000000',
 'stateRoot': HexBytes('0x5a3d14bb5795eea6d6d78c486f86b8d79138fb90658f2c9b3d5c273fc203a86e'),
 'transactionsRoot': HexBytes('0x182eeccbe8ccf72ba9c8857ff3dd048c0506fb802570c711342579aa99401185'),
 'receiptsRoot': HexBytes('0xacc0e2639ea755553beb4b48123fe5d662d7a0d5a30b4f04c99af102ad607faf'),
 'number': 2,
 'gasUsed': 21000,
 'gasLimit': 80000000,
 'extraData': HexBytes('0x'),
 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

Get the transaction information from the block and from the tx itself. 
Note that `gasPrice` may still be `0`, this value is filled with the transaction hasn been executed and included in a block. 

In [74]:
# Tx info:
print(f"maxPriorityFeePerGas\t: {tx['maxPriorityFeePerGas']}\nmaxFeePerGas\t\t: {tx['maxFeePerGas']}\ngas\t\t\t: {tx['gas']}\ngasPrice\t\t: {tx['gasPrice']}")

maxPriorityFeePerGas	: 0
maxFeePerGas		: 1875000000
gas			: 121000
gasPrice		: 875000000


In [75]:
# Block info:
print(f"baseFeePerGas\t\t: {blk['baseFeePerGas']} \ngasLimit\t\t: {blk['gasLimit']}\ngasUsed\t\t\t: {blk['gasUsed']}")

baseFeePerGas		: 875000000 
gasLimit		: 80000000
gasUsed			: 21000


In [76]:
difference_wei / blk['gasUsed'] # the fee price payed per unit of gas

875000000.0

In [77]:
if "anvil" in w3.client_version:
    computed_difference = blk["baseFeePerGas"] * blk['gasUsed']
else:
    computed_difference = tx['maxFeePerGas'] * blk['gasUsed']

In [78]:
computed_difference

18375000000000

In [79]:
difference_wei

18375000000000

In [80]:
# Calculate the difference missing from the sender account: 
assert computed_difference == difference_wei

In [81]:
# Check if this is the amount missing from the sender account: 
assert get_balance(w3.eth.accounts[0],"wei") + computed_difference == 999000000000000000000

In [82]:
# Caluclate the fee paid to the miner: 
fee_for_miner = tx['maxFeePerGas'] - blk['baseFeePerGas']; fee_for_miner

1000000000

In [83]:
wei_for_miner = ( tx['maxFeePerGas'] - blk['baseFeePerGas'] ) * blk['gasUsed']; wei_for_miner

21000000000000

In [84]:
ether_for_miner = w3.from_wei(wei_for_miner,'ether'); ether_for_miner

Decimal('0.000021')

In [85]:
# estimate current base price for a value transfer tx:
CURRENT_BASE_PRICE = float(w3.from_wei(ETH_BASE_FEE_PER_GAS_WEI * 21000,"ether")) * ETH_X_EUR 
print(f"Current base fee price for a value transfer tx in Euro: EUR {CURRENT_BASE_PRICE} €")

Current base fee price for a value transfer tx in Euro: EUR 1.4246209781097947 €


In [86]:
# get the transaction hash (i.e., its ID) of the first (and only) transaction in the block
tx_hash_from_block = blk["transactions"][0].hex()
tx_hash_from_block

'0x93ff785f6eb2fe9ff1ed12ac868d286fc0ea0070f1506c94ef51e797bee274bf'

In [87]:
tx_hash_from_tx = tx["hash"].hex() # hash of the transaction
tx_hash_from_tx

'0x93ff785f6eb2fe9ff1ed12ac868d286fc0ea0070f1506c94ef51e797bee274bf'

In [88]:
assert tx_hash_from_tx == tx_hash_from_block
tx_hash = tx_hash_from_tx

In [89]:
# Query the transaction by its hash is also possible:
tx = w3.eth.get_transaction(tx_hash)
tx

AttributeDict({'hash': HexBytes('0x93ff785f6eb2fe9ff1ed12ac868d286fc0ea0070f1506c94ef51e797bee274bf'),
 'nonce': 0,
 'blockHash': HexBytes('0x076df6ea9e7f09ec7dd186f54c58188299535d5c9a19b66208fb25314775fe33'),
 'blockNumber': 2,
 'transactionIndex': 0,
 'from': '0xf04e3ce2eb267e94463370BaA06ef1F2f8487298',
 'to': '0xd4e5A8028D2d695a9b2262DE611d9B5531C3a2F1',
 'value': 1000000000000000000,
 'gasPrice': 875000000,
 'gas': 121000,
 'input': HexBytes('0x'),
 'v': 1,
 'r': HexBytes('0x54088000a67330b605515b0a422abcf82407f4d80ecf7b94d61501e59af42ccf'),
 's': HexBytes('0x33281a5adc68996460423236b7c5970b80deba5bac0c7a9be01d49b88f8ba66d'),
 'sourceHash': '0x0000000000000000000000000000000000000000000000000000000000000000',
 'isSystemTx': False,
 'type': 2,
 'accessList': [],
 'maxPriorityFeePerGas': 0,
 'maxFeePerGas': 1875000000,
 'chainId': 31337})

In [90]:
# Get the transaction receipt for the respective transaction hash:
tx_receipt = w3.eth.get_transaction_receipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x93ff785f6eb2fe9ff1ed12ac868d286fc0ea0070f1506c94ef51e797bee274bf'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x076df6ea9e7f09ec7dd186f54c58188299535d5c9a19b66208fb25314775fe33'),
 'blockNumber': 2,
 'from': '0xf04e3ce2eb267e94463370BaA06ef1F2f8487298',
 'to': '0xd4e5A8028D2d695a9b2262DE611d9B5531C3a2F1',
 'cumulativeGasUsed': 21000,
 'gasUsed': 21000,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')

* `gasUsed` is the gas used by this transaction
* `cumulativeGasUsed` 
> is the sum of gasUsed by this transaction and all preceding transactions in the same block.

In [91]:
gasUsed_from_block = blk["gasUsed"] # gas used within the whole block
gasUsed_from_block

21000

In [92]:
gasUsed_from_receipt = tx_receipt["gasUsed"] # gas used by the transaction
gasUsed_from_receipt

21000

In [93]:
assert gasUsed_from_block == gasUsed_from_receipt

The transaction was allowed to consue more gas theorectically:

In [94]:
gasAvailable_from_tx = tx["gas"] # theoretical limit of gas available to transaction
gasAvailable_from_tx

121000

In [95]:
assert gasUsed_from_receipt <= gasAvailable_from_tx 

To estimate the gas costs of a tx, without actually executing it you can use:

In [96]:
# estimate the gas price of an transaction without executing it
w3.eth.estimate_gas({'from':w3.eth.accounts[0], 
                    'to':w3.eth.accounts[1], 
                    'value':10**18})

21000

Lets check if the correct amount of gas has been used.

In [97]:
# The actual gas price of the tx
tx_gasPrice = tx["gasPrice"] 
tx_gasPrice

875000000

In [98]:
if "anvil" in w3.client_version:
    assert tx_gasPrice == blk["baseFeePerGas"] 
else:
    assert tx_gasPrice == tx['maxFeePerGas']

In [99]:
def sendEther(_value,_to):
    """ Helper function to quickly send some ether to an address """
    tx_hash = w3.eth.sendTransaction({'from':w3.eth.accounts[0], 
                                      'to':w3.toChecksumAddress(_to), 
                                      'value':w3.toWei(_value,"ether")})
    return tx_hash

# Contract deployment and interaction (the hard way)

Now its time to deploy our first contract.
For the first time we do it the hard way.

## Solidity and its compiler (solc)
Solidity is the high level language:
* https://docs.soliditylang.org/
* https://solidity-by-example.org/

In [100]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.8.23+commit.f704f362.Linux.g++


In [101]:
!solc --help

solc, the Solidity commandline compiler.

This program comes with ABSOLUTELY NO WARRANTY. This is free software, and you
are welcome to redistribute it under certain conditions. See 'solc --license'
for details.

Usage: solc [options] [input_file...]
Compiles the given Solidity input files (or the standard input if "-" is
used as a file name) and outputs the components specified in the options
at standard output or in files in the output directory, if specified.
Imports are automatically read from the filesystem, but it is also possible to
remap paths using the context:prefix=path syntax.
Example:
solc --bin -o /tmp/solcoutput dapp-bin=/usr/local/lib/dapp-bin contract.sol

General Information:
  --help               Show help message and exit.
  --version            Show version and exit.
  --license            Show licensing information and exit.
  --input-file arg     input file

Input Options:
  --base-path path     Use the given path as the root of the source tree 
                

In [102]:
EXAMPLE_PATH="/smartenv/examples/"

In [103]:
%cd {EXAMPLE_PATH}/Greeter 

/smartenv/examples/Greeter


/smartenv/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


* Version Pragma: 
    + https://docs.soliditylang.org/en/v0.8.5/layout-of-source-files.html?highlight=pragma#version-pragma
* SPX-License-Identifier: 
    + https://docs.soliditylang.org/en/v0.8.5/layout-of-source-files.html?highlight=SPX-License#spdx-license-identifier
    + https://spdx.dev/

In [104]:
# execute console command and print Greeter.sol
!cat {EXAMPLE_PATH}/Greeter/Greeter.sol

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.4;

contract Greeter {
    bool private greetbool;
    string public greeting;
    uint256 public greetbit;

    constructor() {
        greeting = 'Hello';
    }

    function setGreeting(string memory _greeting) public {
        greeting = _greeting;
    }

    function getGreeting() view public returns (string memory) {
        return greeting;
    }

    function setGreetbool(bool _bool) public {
        greetbool = _bool;
    }

    function getGreetbool() view public returns (bool) {
        return greetbool; 
    }

    function setGreetbit(uint256 _bit) public {
        greetbit = _bit;
    }

    fallback() external payable {
        greetbit = greetbit ^ 1;
    }

    receive() external payable {
        greetbool = !greetbool;
    }
}


In [105]:
!solc --version

solc, the solidity compiler commandline interface
Version: 0.8.23+commit.f704f362.Linux.g++


In [106]:
!ls /usr/local/bin/solc*

/usr/local/bin/solc	 /usr/local/bin/solc_7.4   /usr/local/bin/solc_8.4
/usr/local/bin/solc_5.4  /usr/local/bin/solc_8.23


In [107]:
!solc --bin --overwrite -o ./ Greeter.sol

Compiler run successful. Artifact(s) can be found in directory "./".


In [108]:
!cat Greeter.bin

608060405234801562000010575f80fd5b506040518060400160405280600581526020017f48656c6c6f00000000000000000000000000000000000000000000000000000081525060019081620000579190620002c2565b50620003a6565b5f81519050919050565b7f4e487b71000000000000000000000000000000000000000000000000000000005f52604160045260245ffd5b7f4e487b71000000000000000000000000000000000000000000000000000000005f52602260045260245ffd5b5f6002820490506001821680620000da57607f821691505b602082108103620000f057620000ef62000095565b5b50919050565b5f819050815f5260205f209050919050565b5f6020601f8301049050919050565b5f82821b905092915050565b5f60088302620001547fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff8262000117565b62000160868362000117565b95508019841693508086168417925050509392505050565b5f819050919050565b5f819050919050565b5f620001aa620001a46200019e8462000178565b62000181565b62000178565b9050919050565b5f819050919050565b620001c5836200018a565b620001dd620001d482620001b1565b84845462000123565b825550505050565b5f90565b620001f3620001e5565b

In [109]:
# assign cat output to python variable
out = !cat Greeter.bin
cbin = out[0]

In [110]:
!solc --abi --overwrite -o ./ Greeter.sol

Compiler run successful. Artifact(s) can be found in directory "./".


In [111]:
!cat Greeter.abi

[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"stateMutability":"payable","type":"fallback"},{"inputs":[],"name":"getGreetbool","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getGreeting","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"greetbit","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"greeting","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_bit","type":"uint256"}],"name":"setGreetbit","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bool","name":"_bool","type":"bool"}],"name":"setGreetbool","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"interna

In [112]:
out = !cat Greeter.abi
cabi = out[0]

## Deploy contract

One way to send a deploy transaction is using the web3py libary helper function cunstructor.
This requires:
* The ABI of the contract
* The bytecode of the contract

In [113]:
# send transaction to deploy contract with helper function
contract=w3.eth.contract(abi=cabi,
                         bytecode=cbin)
tx_hash = contract.constructor().transact({"from":w3.eth.accounts[0],})
tx_hash

HexBytes('0x7634700a893eeb4a43d606ce2e391cbd8a9dc774123a383930f2a2ac1ee60f72')

Another way which can also estimate the gas costs of the deploy transaction is sending it 
directly via `sendTransaction` and add the previously compiled contract as `data`. This requires:
* The bytecode of the contract

In [114]:
# estimate gas of deployment transaction
w3.eth.estimate_gas({"from":w3.eth.accounts[0],"data":cbin,"value":0})

611435

In [115]:
# send transaction and get transaction hash
tx_hash = w3.eth.send_transaction({"from":w3.eth.accounts[0],"data":cbin,"value":0})
tx_hash

HexBytes('0xabac8c14474bab8b16e7037c24c8c9087be8b0ab52917d2c8c80662588c9db81')

In [116]:
# here you see the compiled contract as data in the 'input' field 
# note that the 'to' address is 'None' in smart contract depoloyments
tx_info = w3.eth.get_transaction(tx_hash)
tx_info

AttributeDict({'hash': HexBytes('0xabac8c14474bab8b16e7037c24c8c9087be8b0ab52917d2c8c80662588c9db81'),
 'nonce': 2,
 'blockHash': HexBytes('0x587212673313a5a10f91a0e33149cbd49067229a602825af8c72400843726968'),
 'blockNumber': 4,
 'transactionIndex': 0,
 'from': '0xf04e3ce2eb267e94463370BaA06ef1F2f8487298',
 'to': None,
 'value': 0,
 'gasPrice': 671435135,
 'gas': 711435,
 'input': HexBytes('0x608060405234801562000010575f80fd5b506040518060400160405280600581526020017f48656c6c6f00000000000000000000000000000000000000000000000000000081525060019081620000579190620002c2565b50620003a6565b5f81519050919050565b7f4e487b71000000000000000000000000000000000000000000000000000000005f52604160045260245ffd5b7f4e487b71000000000000000000000000000000000000000000000000000000005f52602260045260245ffd5b5f6002820490506001821680620000da57607f821691505b602082108103620000f057620000ef62000095565b5b50919050565b5f819050815f5260205f209050919050565b5f6020601f8301049050919050565b5f82821b905092915050565b5f60088302620001547f

In [117]:
tx_info["to"]

Normally one has to wait till the transaction goes through, with ganache this is almost instant 

In [118]:
# wait till tx goes through 
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
tx_receipt

# other way without wait only successful if mined:
# w3.eth.getTransactionReceipt(tx_hash)

AttributeDict({'transactionHash': HexBytes('0xabac8c14474bab8b16e7037c24c8c9087be8b0ab52917d2c8c80662588c9db81'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x587212673313a5a10f91a0e33149cbd49067229a602825af8c72400843726968'),
 'blockNumber': 4,
 'from': '0xf04e3ce2eb267e94463370BaA06ef1F2f8487298',
 'to': None,
 'cumulativeGasUsed': 611435,
 'gasUsed': 611435,
 'contractAddress': '0x788A88A039F4ea7002b7ece27B43B1C1E9cdEE87',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [119]:
# the important thing from the tx receipt is the address at which 
# the contract was deployed
caddress = tx_receipt["contractAddress"]
caddress

'0x788A88A039F4ea7002b7ece27B43B1C1E9cdEE87'

In [120]:
# double check if the gasUsed was really our estimate
blk = w3.eth.get_block(tx_receipt["blockNumber"])
blk["gasUsed"]

611435

## Interact with deployed contract

To get an instance of a deployed contract at its current state we require:
* The contract address `caddress`
* Some ABI information `cabi` either from compiling the contract like done previously or reconstructed through reverse engineering of the bytecode stored in the blockchain.  

In [121]:
# instanciate a deployed contract
instance = w3.eth.contract(address=caddress,
                           abi=cabi)

In [122]:
instance.address # The address of the deployed contract

'0x788A88A039F4ea7002b7ece27B43B1C1E9cdEE87'

In [123]:
instance.abi # The ABI of the deplyed contract

[{'inputs': [], 'stateMutability': 'nonpayable', 'type': 'constructor'},
 {'stateMutability': 'payable', 'type': 'fallback'},
 {'inputs': [],
  'name': 'getGreetbool',
  'outputs': [{'internalType': 'bool', 'name': '', 'type': 'bool'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'name': 'getGreeting',
  'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'name': 'greetbit',
  'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'name': 'greeting',
  'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [{'internalType': 'uint256', 'name': '_bit', 'type': 'uint256'}],
  'name': 'setGreetbit',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'},
 {'inputs': [{'internalType': 'bool', '

### Read storage from deployed contract

In [124]:
# view functions dont change state (just read it) and can therefore be executed to read state
# without performing a transaction on the blockchain
print(instance.functions.getGreeting().call())
print(instance.functions.getGreetbool().call())

Hello
False


In [125]:
# the value of public variables can also be read directly without a getter function
print(instance.functions.greeting().call())
print(instance.functions.greetbit().call())

Hello
0


In [126]:
# the value of private variables cannot be queried directly,
# but this does not offer protection against reads 
import sys
try:
    print(instance.functions.greetbool().call())
except:
    e = sys.exc_info()[0]
    print(e)

<class 'web3.exceptions.ABIFunctionNotFound'>


To read the first value from the storage (in our case the bool variable) we can use
the following function (even if there would be not getter function for it):

In [127]:
assert instance.functions.getGreetbool().call() == False
print(instance.functions.getGreetbool().call())

False


In [128]:
w3.eth.get_storage_at(caddress,0).hex() # uninitialized / None

'0x0000000000000000000000000000000000000000000000000000000000000000'

In [129]:
if "ganache" in w3.client_version:
    # in ganache uninitialized is just "0x"
    assert w3.eth.get_storage_at(caddress,0).hex() == "0x" 
else:
    assert w3.eth.get_storage_at(caddress,0).hex().endswith("0")
w3.eth.get_storage_at(caddress,0)

HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000')

No we execute a transaction and see if the bool value has changed:

In [130]:
tx_hash = instance.functions.setGreetbool(True).transact({"from":w3.eth.accounts[0]})
print("Tx hash: ",tx_hash.hex())
print("Now lets wait till tx gets mined (in ganache this is instant) ... ")
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

Tx hash:  0x78a0c54005812a7357774b6b5b13f646cb0ba63a3e6b901e72069538a59f142d
Now lets wait till tx gets mined (in ganache this is instant) ... 


In [131]:
assert instance.functions.getGreetbool().call() == True
print(instance.functions.getGreetbool().call())

True


In [132]:
assert int(w3.eth.get_storage_at(caddress,0).hex()[2:],16) == 0x01
w3.eth.get_storage_at(caddress,0)

HexBytes('0x0000000000000000000000000000000000000000000000000000000000000001')

In [133]:
w3.eth.get_storage_at(caddress,0).hex()

'0x0000000000000000000000000000000000000000000000000000000000000001'

### Function input (calldata)

Now lets set the string value:

In [134]:
# Read the current greeting string value
print(instance.functions.greeting().call())

Hello


In [135]:
tx_hash = instance.functions.setGreeting("Hello World!").transact(
    {"from":w3.eth.accounts[0]})
tx_hash.hex()

'0x176461352dee81a2cbae46fed735da727aaeec9943459486cceb90de125c21c7'

In [136]:
# lets check if the output is now different i.e., the value has changed 
print(instance.functions.greeting().call())

Hello World!


In [137]:
tx_info = w3.eth.get_transaction(tx_hash)
tx_info

AttributeDict({'hash': HexBytes('0x176461352dee81a2cbae46fed735da727aaeec9943459486cceb90de125c21c7'),
 'nonce': 4,
 'blockHash': HexBytes('0xac56ee3f386ee85ca49cb4c11881245553059ffbe0d1cb1be8465c1a91854af1'),
 'blockNumber': 6,
 'transactionIndex': 0,
 'from': '0xf04e3ce2eb267e94463370BaA06ef1F2f8487298',
 'to': '0x788A88A039F4ea7002b7ece27B43B1C1E9cdEE87',
 'value': 0,
 'gasPrice': 515270684,
 'gas': 128183,
 'input': HexBytes('0xa41368620000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000c48656c6c6f20576f726c64210000000000000000000000000000000000000000'),
 'v': 1,
 'r': HexBytes('0x4badeab5cf5a8049b8804b7ee98bde3b0894841a6cbdcada81780b8410169330'),
 's': HexBytes('0x32912303198bbad0d96c80bc079cd88d056a670f6e9cfa235b7214b69c9ccd4f'),
 'sourceHash': '0x0000000000000000000000000000000000000000000000000000000000000000',
 'isSystemTx': False,
 'type': 2,
 'accessList': [],
 'maxPriorityFeePerGas': 0,
 'maxFeePerGa

In [168]:
input_data = tx_info["input"]
input_data

HexBytes('0xa41368620000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000c48656c6c6f20576f726c64210000000000000000000000000000000000000000')

In [169]:
input_data.hex()

'0xa41368620000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000c48656c6c6f20576f726c64210000000000000000000000000000000000000000'

In [170]:
bytes.fromhex(input_data.hex()[2:])

b'\xa4\x13hb\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0cHello World!\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [171]:
w3.to_bytes(hexstr=input_data.hex()) # w3 build in function

b'\xa4\x13hb\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0cHello World!\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

Lets decode the calldata input, in this case of one dynamic type parameter string:
* https://solidity.readthedocs.io/en/latest/abi-spec.html#abi

* The first 4 bytes (8 hex characters) are the **function signature**. 
See next section for more details. 

* The remaining bytes are the string parameter, seperated and aligned in three 32 bytes (256 bit) chunks. 

In [172]:
input_data_bytes = bytes.fromhex(input_data.hex()[2:])

In [179]:
input_data_bytes

b'\xa4\x13hb\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0cHello World!\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [188]:
len(input_data_bytes)

100

In [192]:
assert (len(input_data_bytes)-4)/32 == 3.0

In [173]:
input_data_bytes[:4] # function signature

b'\xa4\x13hb'

In [193]:
(len(input_data_bytes)-4)//32

3

In [195]:
# slice input data bytes inptu 32 byte (256 bit) chunks
dynamic_parameter = [ input_data_bytes[4:][i:i+32].hex() for i in range(0,len(input_data_bytes[4:]),32) ]
dynamic_parameter

['0000000000000000000000000000000000000000000000000000000000000020',
 '000000000000000000000000000000000000000000000000000000000000000c',
 '48656c6c6f20576f726c64210000000000000000000000000000000000000000']

**Dynamic parameter:**
The first chunk/parameter is the offset in bytes to the data part of the first (and only) dynamic parameter string in our case. 
This offset is computed starting from the arguments block. 

In [197]:
assert int(dynamic_parameter[0],16) == 0x20
offset = 0x20
offset

32

After the offset the length of the string is encoded. 

In [198]:
assert int(dynamic_parameter[1],16) == 0x0c
string_length = 0x0c
string_length

12

The rest is the actual string

In [209]:
assert bytes.fromhex(dynamic_parameter[2])[:int(dynamic_parameter[1],16)].decode("utf-8") == "Hello World!"

In [210]:
# lets decode the input data of the transaction
input_data = bytes.fromhex("48656c6c6f20576f726c6421").decode("utf-8")
print(input_data) 
print("length = ",len(input_data))

Hello World!
length =  12


There is also helper function that can decode and encode function parameters in a more human readable form:

In [211]:
# decode
instance.decode_function_input(tx_info.input)

(<Function setGreeting(string)>, {'_greeting': 'Hello World!'})

In [212]:
# encode
payload= instance.encodeABI(fn_name='setGreeting', args=["Hello World!"])
payload

'0xa41368620000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000000c48656c6c6f20576f726c64210000000000000000000000000000000000000000'

There is also a way to directly access the variables in the storage of a contract. This also required decoding which is slightly different:
> For short byte arrays, they store their data in the same slot where the length is also stored. In particular: if the data is at most 31 bytes long, it is stored in the higher-order bytes (left aligned) and the lowest-order byte stores length * 2

* https://solidity.readthedocs.io/en/latest/miscellaneous.html?highlight=storage#bytes-and-string

The first element in storage is the bool variable (we have seen this already):

In [145]:
w3.eth.get_storage_at(caddress,0)

HexBytes('0x0000000000000000000000000000000000000000000000000000000000000001')

The second element in storage is the string:

In [146]:
w3.eth.get_storage_at(caddress,1)

HexBytes('0x48656c6c6f20576f726c64210000000000000000000000000000000000000018')

In [147]:
hex(string_length * 2) # lowest-order byte stores the length * 2

'0x18'

### Function signature (aka. method ID)
The four byte *function signature* or *method id* has to be specified at the beginning of the input data. It selects the function that is invoked with the respective arguments.

In [148]:
# 4 byte function signature or method_id
len("a4136862")//2

4

In [149]:
from sha3 import keccak_256
sha3_hash = keccak_256(b"setGreeting(string)").hexdigest()
method_id = sha3_hash[:8]
method_id

'a4136862'

In [150]:
instance.functions.greetbit().call()

0

In [151]:
w3.eth.get_storage_at(caddress,2)

HexBytes('0x')

In [152]:
tx_hash = instance.functions.setGreetbit(1).transact(
    {"from":w3.eth.accounts[0]})
tx_hash

HexBytes('0x7f259ff18fdc045ef0f0460646b9de19cae08246b13887fcaa7a0dce7e567cf1')

In [153]:
instance.functions.greetbit().call()

1

In [154]:
w3.eth.get_storage_at(caddress,2)

HexBytes('0x0000000000000000000000000000000000000000000000000000000000000001')

In [155]:
tx_info = w3.eth.get_transaction(tx_hash)
tx_info['input']

HexBytes('0x4f11dbea0000000000000000000000000000000000000000000000000000000000000001')

In [156]:
from sha3 import keccak_256
sha3_hash = keccak_256(b"setGreetbit(uint256)").hexdigest()
method_id = sha3_hash[:8]
method_id

'4f11dbea'

In [157]:
len("0000000000000000000000000000000000000000000000000000000000000001")/2 * 8

256.0

You can also manually send a transaction, this should work with `geth`

In [158]:
# This should be possible with geth but is not with web3py:
cdata = b"0x4f11dbea0000000000000000000000000000000000000000000000000000000000000000"
tx_hash = w3.eth.send_transaction({"from":w3.eth.accounts[0],"to":caddress,"data":cdata,"value":0,"gas":100_000})
tx_hash

HexBytes('0x2577c21d0c7bd3e162ffb3ea5400b774d2aae2757690857d26e6254f4ded625e')

In [159]:
instance.functions.greetbit().call()

0

You can also use:
* the `getData()` method in the `geth` REPL console to construct the `input` data hex string. 
* the `encodeABI()` method in `web3`

In [160]:
payload= instance.encodeABI(fn_name='setGreetbit', args=[1])
payload

'0x4f11dbea0000000000000000000000000000000000000000000000000000000000000001'

### Debug Transaction

To debug a transaction the following functions can be used: 

In [161]:
# debug i.e. trace a transaction and its operations 
w3.provider.make_request("debug_traceTransaction", [tx_hash.hex(), {}])

{'id': 155,
 'jsonrpc': '2.0',
 'result': {'gas': '0x6ad9',
  'structLogs': [{'depth': 1,
    'error': '',
    'gas': '0x12ff8',
    'gasCost': 3,
    'memory': [],
    'op': 'PUSH1',
    'pc': 0,
    'stack': [],
    'storage': {}},
   {'depth': 1,
    'error': '',
    'gas': '0x12ff5',
    'gasCost': 3,
    'memory': [],
    'op': 'PUSH1',
    'pc': 2,
    'stack': ['0000000000000000000000000000000000000000000000000000000000000080'],
    'storage': {}},
   {'depth': 1,
    'error': '',
    'gas': '0x12ff2',
    'gasCost': 12,
    'memory': ['0000000000000000000000000000000000000000000000000000000000000000',
     '0000000000000000000000000000000000000000000000000000000000000000',
     '0000000000000000000000000000000000000000000000000000000000000000'],
    'op': 'MSTORE',
    'pc': 4,
    'stack': ['0000000000000000000000000000000000000000000000000000000000000080',
     '0000000000000000000000000000000000000000000000000000000000000040'],
    'storage': {}},
   {'depth': 1,
    'error'

In [162]:
# debug the old way:
#w3.providers[0].make_request("debug_traceTransaction", [tx_hash.hex(), {}])

### Fallback and receive functions

The **fallback function** is the function that is invoked when the method ID does not match any other function, and some `data` is provided as parameter (or not *receive function* is defined and the *fallback function* is payable). 
For a contract to be able to accept ether this way, the fallback function has to be defined as `payable`. 
* https://docs.soliditylang.org/en/v0.8.5/contracts.html?highlight=fallback#fallback-function

> A payable fallback function is also executed for plain Ether transfers, if no receive Ether function is present.  It is recommended to always define a receive Ether function as well, if you define a payable fallback function to distinguish Ether transfers from interface confusions.

In [163]:
instance.functions.greetbit().call()

0

In [164]:
instance.functions.getGreetbool().call()

True

In [165]:
get_balance(instance.address)

0

In [166]:
cdata = b"0xffffffff" # some random data, will be ignored anyway
tx_hash = w3.eth.send_transaction({"from":w3.eth.accounts[0],"to":caddress,"data":cdata,"value":1*10**18})
tx_hash

HexBytes('0x6efe36051cd931bd4197143e4bcbe65cf9c8b22bc670f95d3a33f5ecbc0d3317')

In [167]:
instance.functions.greetbit().call()

1

In [168]:
get_balance(instance.address)

Decimal('1')

In [169]:
instance.functions.getGreetbool().call()

True

The **receive function** is the function that is invoked when the method ID does not match any other function, and **no** `data` is provided as parameter, but value transferred. 
The receive function has to be defined as `payable`. The receive function is not necessary!
* https://docs.soliditylang.org/en/v0.8.5/contracts.html?highlight=fallback#receive-ether-function

> Contracts that receive Ether directly (without a function call, i.e. using send or transfer) but do not define a receive Ether function or a payable fallback function throw an exception, sending back the Ether (this was different before Solidity v0.4.0). So if you want your contract to receive Ether, you have to implement a receive Ether function (using payable fallback functions for receiving Ether is not recommended, since it would not fail on interface confusions).

* **Warning 1:**
> In the worst case, the receive function can only rely on 2300 gas being available (for example when send or transfer is used), leaving little room to perform other operations except basic logging. The following operations will consume more gas than the 2300 gas stipend:
    + Writing to storage
    + Creating a contract
    + Calling an external function which consumes a large amount of gas
    + Sending Ether

* **Warning 2:** 
> A contract without a receive Ether function can receive Ether as a recipient of a coinbase transaction (aka miner block reward) or as a destination of a selfdestruct.
A contract cannot react to such Ether transfers and thus also cannot reject them. This is a design choice of the EVM and Solidity cannot work around it.
It also means that address(this).balance can be higher than the sum of some manual accounting implemented in a contract (i.e. having a counter updated in the receive Ether function).

In [170]:
instance.functions.greetbit().call()

1

In [171]:
instance.functions.getGreetbool().call()

True

In [172]:
tx_hash = w3.eth.send_transaction({"from":w3.eth.accounts[0],"to":caddress,"value":1*10**18})
tx_hash

HexBytes('0x1076bf56a51ee85a65fa4f666fc25d0248b5d6d920ffd272c90943664e1adc52')

In [173]:
tx = w3.eth.get_transaction(tx_hash)
tx["gas"]

126254

In [174]:
tx_receipt = w3.eth.get_transaction_receipt(tx_hash)
tx_receipt["gasUsed"]

21454

In [175]:
instance.functions.getGreetbool().call()

False

In [176]:
instance.functions.greetbit().call()

1

#### Sending and recieveing ether to contracts
https://solidity-by-example.org/sending-ether/


# Contract deployment and interaction 

Using a small utility script

In [177]:
cat {EXAMPLE_PATH}/Greeter/Greeter.sol

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.4;

contract Greeter {
    bool private greetbool;
    string public greeting;
    uint256 public greetbit;

    constructor() {
        greeting = 'Hello';
    }

    function setGreeting(string memory _greeting) public {
        greeting = _greeting;
    }

    function getGreeting() view public returns (string memory) {
        return greeting;
    }

    function setGreetbool(bool _bool) public {
        greetbool = _bool;
    }

    function getGreetbool() view public returns (bool) {
        return greetbool; 
    }

    function setGreetbit(uint256 _bit) public {
        greetbit = _bit;
    }

    fallback() external payable {
        greetbit = greetbit ^ 1;
    }

    receive() external payable {
        greetbool = !greetbool;
    }
}


In [178]:
import util as util
import importlib
importlib.reload(util)

<module 'util' from '/smartenv/course/util/__init__.py'>

In [179]:
%cd /

/


In [180]:
!pwd

/


In [181]:
# !solc --combined-json "abi,bin" /smartenv/examples/Greeter/Greeter.sol

In [182]:
util.connect(host="172.18.0.2",port=8545,poa=False)
# Deployment and instanciation:
instance = util.compile_and_deploy_contract(EXAMPLE_PATH + "Greeter/Greeter.sol",compiler="solc")

In [183]:
instance.address

'0x0290FB167208Af455bB137780163b7B7a9a10C16'

In [184]:
instance.functions.getGreeting().call()

'Hello'

In [185]:
# transaction without value and without manual gas specification
# wait till tx is mined
tx_receipt = instance.functions.setGreeting("Nihao").transact=({"from": w3.eth.accounts[0]})
tx_receipt

{'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1'}

In [186]:
instance.functions.getGreeting().call()

'Hello'

In [187]:
# Re-deployment and instanciation:
instance = util.compile_and_deploy_contract(EXAMPLE_PATH + "Greeter/Greeter.sol",compiler="solc")

In [188]:
instance.address

'0x9b1f7F645351AF3631a656421eD2e40f2802E6c0'

In [189]:
instance.functions.getGreeting().call()

'Hello'

In [190]:
# verbose transaction with manually specified gas and potentially value transferred to contract 
tx_hash = instance.functions.setGreeting("Nihao").transact({"from":w3.eth.accounts[0],
                                                            "value":0,
                                                            "gas":1_000_000})
tx_hash

HexBytes('0xdbc584ff041599a8088404cfe4efbcef98809f9c7fd5e36f7b0794b182d75748')

In [191]:
instance.functions.getGreeting().call()

'Nihao'

In [192]:
# Verbose instanciation of already deployed contract (ABI generated from soruce): 
instance = util.get_contract_instance(instance.address,None,path=EXAMPLE_PATH + "Greeter/Greeter.sol",compiler="solc")

In [193]:
instance.functions.getGreeting().call()

'Nihao'

# Events

Events are a way to notify clients listening to new blocks about smart contract operations that have been performed.
* https://solidity.readthedocs.io/en/develop/contracts.html#events
* https://web3py.readthedocs.io/en/stable/filters.html?highlight=events
* https://web3py.readthedocs.io/en/stable/contracts.html#events

In [194]:
!cat {EXAMPLE_PATH}/Eventer/Eventer.sol

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.4;

contract Eventer {

    event create(uint256 v);
    event funcall(address caller,uint256 value);
    event funcall2(address indexed caller,uint256 indexed value);
    event fallcall(address caller);

    constructor() {
        emit create(1);
    }

    function func1(uint256 v) public {
        emit funcall(msg.sender,v);
    }

    function func2(uint256 v) public {
        emit funcall2(msg.sender,v);
    }

    receive() external payable{
        emit fallcall(msg.sender);
    }

}


In [195]:
!solc --bin {EXAMPLE_PATH}/Eventer/Eventer.sol


======= smartenv/examples/Eventer/Eventer.sol:Eventer =======
Binary:
608060405234801561000f575f80fd5b507f780900dcfb922770b66b73546245c9d725e14dd206326f4f8f5a706976c2b61d60016040516100409190610098565b60405180910390a16100b1565b5f819050919050565b5f819050919050565b5f819050919050565b5f61008261007d6100788461004d565b61005f565b610056565b9050919050565b61009281610068565b82525050565b5f6020820190506100ab5f830184610089565b92915050565b610267806100be5f395ff3fe60806040526004361061002c575f3560e01c806316d93ade1461006e578063254e43db146100965761006a565b3661006a577f558048e62c62b6dedcadcb7b740131fd129cfa4019d5a5158a5befc2976d0f1d336040516100609190610180565b60405180910390a1005b5f80fd5b348015610079575f80fd5b50610094600480360381019061008f91906101d0565b6100be565b005b3480156100a1575f80fd5b506100bc60048036038101906100b791906101d0565b610105565b005b803373ffffffffffffffffffffffffffffffffffffffff167f1a7f26d3c848a605ab293135411d69990f60fa592874ca544b2f290579fbe1b860405160405180910390a350565b7fcdb58a22429d89e572d1733

In [196]:
# create a contract instance 
einstance = util.compile_and_deploy_contract(EXAMPLE_PATH + "Eventer/Eventer.sol",compiler="solc",w3conn=w3)

In [197]:
einstance.address

'0x2612Af3A521c2df9EAF28422Ca335b04AdF3ac66'

Now we create a listener to listen for the latest *create* events and return all of them.

In [198]:
event_filter = einstance.events.create.create_filter(fromBlock='latest')

In [199]:
event_filter.get_all_entries()

[AttributeDict({'args': AttributeDict({'v': 1}),
  'event': 'create',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xed2a6a4445cd4527d1c949c245a229c00fff88e16c4ed412acbae9131e7e1f9a'),
  'address': '0x2612Af3A521c2df9EAF28422Ca335b04AdF3ac66',
  'blockHash': HexBytes('0x55004210089d918e0bdeacb53dc18cbe70acc6732d90da60f0e742d021470673'),
  'blockNumber': 14})]

Now we fire a new event by creating a new transaction and keep the `tx_receipt` which we cen process later on. 

In [200]:
tx_hash = einstance.functions.func1(3).transact({"from":w3.eth.accounts[0]})
print(tx_hash.hex())
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
tx_receipt

0x73feb8f564343cd139aec5e605ba72aac055822b20323fdd8429d7e92fe3ff53


AttributeDict({'transactionHash': HexBytes('0x73feb8f564343cd139aec5e605ba72aac055822b20323fdd8429d7e92fe3ff53'),
 'transactionIndex': 0,
 'blockNumber': 15,
 'blockHash': HexBytes('0x2628085b44a129123fb4aa6c9b7edfd46a318ad907f3f572c1cc8327b92b13d3'),
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0x2612Af3A521c2df9EAF28422Ca335b04AdF3ac66',
 'cumulativeGasUsed': 23198,
 'gasUsed': 23198,
 'contractAddress': None,
 'logs': [AttributeDict({'address': '0x2612Af3A521c2df9EAF28422Ca335b04AdF3ac66',
   'blockHash': HexBytes('0x2628085b44a129123fb4aa6c9b7edfd46a318ad907f3f572c1cc8327b92b13d3'),
   'blockNumber': 15,
   'data': HexBytes('0x00000000000000000000000090f8bf6a479f320ead074411a4b0e7944ea8c9c10000000000000000000000000000000000000000000000000000000000000003'),
   'logIndex': 0,
   'removed': False,
   'topics': [HexBytes('0xcdb58a22429d89e572d17333db36ab8054687b985157808a3aefd24cdbe17520')],
   'transactionHash': HexBytes('0x73feb8f564343cd139aec5e605ba72aac055822b203

In [201]:
# concise way, not further supported with events:
#tx_receipt = einstance.func1(3,transact={"from":w3.eth.accounts[0]})
#tx_receipt

The `tx_receipt` contains the respective event.

In [202]:
einstance.events.funcall().process_receipt(tx_receipt)

(AttributeDict({'args': AttributeDict({'caller': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
   'value': 3}),
  'event': 'funcall',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0x73feb8f564343cd139aec5e605ba72aac055822b20323fdd8429d7e92fe3ff53'),
  'address': '0x2612Af3A521c2df9EAF28422Ca335b04AdF3ac66',
  'blockHash': HexBytes('0x2628085b44a129123fb4aa6c9b7edfd46a318ad907f3f572c1cc8327b92b13d3'),
  'blockNumber': 15}),)

The `tx_receipt` does **not** contain the respective event.

In [203]:
einstance.events.funcall2().process_receipt(tx_receipt)

/smartenv/venv/lib/python3.10/site-packages/web3/contract/base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0x73feb8f564343cd139aec5e605ba72aac055822b20323fdd8429d7e92fe3ff53') and logIndex: 0 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(


()

Now lets fire the `funcall` event again collect all events until now

In [204]:
tx_hash = einstance.functions.func1(4).transact({"from":w3.eth.accounts[0]})
print(tx_hash.hex())
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
tx_receipt

0x517f6beeb2886b804599842c6729a9fc20a9254326aed77a35177b15d7170ff1


AttributeDict({'transactionHash': HexBytes('0x517f6beeb2886b804599842c6729a9fc20a9254326aed77a35177b15d7170ff1'),
 'transactionIndex': 0,
 'blockNumber': 16,
 'blockHash': HexBytes('0x27feaee42d3e09551bf154a4a207d6c819e0ffd1607f107afdf5b54d3231f411'),
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0x2612Af3A521c2df9EAF28422Ca335b04AdF3ac66',
 'cumulativeGasUsed': 23198,
 'gasUsed': 23198,
 'contractAddress': None,
 'logs': [AttributeDict({'address': '0x2612Af3A521c2df9EAF28422Ca335b04AdF3ac66',
   'blockHash': HexBytes('0x27feaee42d3e09551bf154a4a207d6c819e0ffd1607f107afdf5b54d3231f411'),
   'blockNumber': 16,
   'data': HexBytes('0x00000000000000000000000090f8bf6a479f320ead074411a4b0e7944ea8c9c10000000000000000000000000000000000000000000000000000000000000004'),
   'logIndex': 0,
   'removed': False,
   'topics': [HexBytes('0xcdb58a22429d89e572d17333db36ab8054687b985157808a3aefd24cdbe17520')],
   'transactionHash': HexBytes('0x517f6beeb2886b804599842c6729a9fc20a9254326a

In [205]:
event_filter = einstance.events.funcall.create_filter(fromBlock=0)

In [206]:
event_filter.get_all_entries()

[AttributeDict({'args': AttributeDict({'caller': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
   'value': 3}),
  'event': 'funcall',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0x73feb8f564343cd139aec5e605ba72aac055822b20323fdd8429d7e92fe3ff53'),
  'address': '0x2612Af3A521c2df9EAF28422Ca335b04AdF3ac66',
  'blockHash': HexBytes('0x2628085b44a129123fb4aa6c9b7edfd46a318ad907f3f572c1cc8327b92b13d3'),
  'blockNumber': 15}),
 AttributeDict({'args': AttributeDict({'caller': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
   'value': 4}),
  'event': 'funcall',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0x517f6beeb2886b804599842c6729a9fc20a9254326aed77a35177b15d7170ff1'),
  'address': '0x2612Af3A521c2df9EAF28422Ca335b04AdF3ac66',
  'blockHash': HexBytes('0x27feaee42d3e09551bf154a4a207d6c819e0ffd1607f107afdf5b54d3231f411'),
  'blockNumber': 16})]

In [207]:
tx_hash = w3.eth.send_transaction({"from":w3.eth.accounts[0],
                                  "to":einstance.address,
                                  "value":1,
                                  "gas":1_000_000})
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x8e3f106b9469c63012d423620a35bdb3bb922c410eea0dd3c247c167da6d1382'),
 'transactionIndex': 0,
 'blockNumber': 17,
 'blockHash': HexBytes('0xb6d0e452a780fb7e3d9a7ddcd98d6f5a890c7707032758d099be01f42cdf5d24'),
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0x2612Af3A521c2df9EAF28422Ca335b04AdF3ac66',
 'cumulativeGasUsed': 22290,
 'gasUsed': 22290,
 'contractAddress': None,
 'logs': [AttributeDict({'address': '0x2612Af3A521c2df9EAF28422Ca335b04AdF3ac66',
   'blockHash': HexBytes('0xb6d0e452a780fb7e3d9a7ddcd98d6f5a890c7707032758d099be01f42cdf5d24'),
   'blockNumber': 17,
   'data': HexBytes('0x00000000000000000000000090f8bf6a479f320ead074411a4b0e7944ea8c9c1'),
   'logIndex': 0,
   'removed': False,
   'topics': [HexBytes('0x558048e62c62b6dedcadcb7b740131fd129cfa4019d5a5158a5befc2976d0f1d')],
   'transactionHash': HexBytes('0x8e3f106b9469c63012d423620a35bdb3bb922c410eea0dd3c247c167da6d1382'),
   'transactionIndex': 0})],
 'logsBloo

In [208]:
# get the fallback function event
einstance.events.fallcall().process_receipt(tx_receipt)

(AttributeDict({'args': AttributeDict({'caller': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1'}),
  'event': 'fallcall',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0x8e3f106b9469c63012d423620a35bdb3bb922c410eea0dd3c247c167da6d1382'),
  'address': '0x2612Af3A521c2df9EAF28422Ca335b04AdF3ac66',
  'blockHash': HexBytes('0xb6d0e452a780fb7e3d9a7ddcd98d6f5a890c7707032758d099be01f42cdf5d24'),
  'blockNumber': 17}),)

# Visibility

Generally all data stored in the blockchain is visible!
Function invocations can be restricted but be careful!

* https://solidity.readthedocs.io/en/v0.4.25/contracts.html#visibility-and-getters

In [209]:
!cat {EXAMPLE_PATH}/Visibility/Visibility.sol

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.4;

contract Visibility {
    uint256 constant public cpub_int = 0x10; //set at compile time
    uint256 public pub_int;   //getter is created automatically 
    uint256 internal int_int; //no getter but available in derived contracts 
    uint256 private priv_int;

    event echange(uint256 indexed v);
    event ichange(uint256 indexed v);
    event pchange(uint256 indexed v);

    constructor() payable {
        pub_int = 0x20;
        int_int = 0x30;
        priv_int = 0x40;
    }
    
    // Function _should_ only be called from external accounts/contracts
    // Although can also be called form this contract with this.
    function ext_change() external returns (uint256) {
        priv_int += 1;
        emit echange(priv_int);
        return priv_int;
    }
    
    // Internal functions cannot be called directly from external accounts/contracts
    // Only indirectly 
    function int_change() internal returns (uint256) {
     

In [210]:
!solc --bin {EXAMPLE_PATH}/Visibility/Visibility.sol


======= smartenv/examples/Visibility/Visibility.sol:Visibility =======
Binary:
608060405260205f8190555060306001819055506040600281905550610437806100285f395ff3fe608060405234801561000f575f80fd5b5060043610610060575f3560e01c80630f66c3d2146100645780634c370d2514610094578063529f246f146100b2578063ebd3b3dd146100d0578063f455a815146100ee578063fcce02071461010c575b5f80fd5b61007e6004803603810190610079919061030f565b61013c565b60405161008b9190610349565b60405180910390f35b61009c610152565b6040516100a99190610349565b60405180910390f35b6100ba6101a3565b6040516100c79190610349565b60405180910390f35b6100d8610269565b6040516100e59190610349565b60405180910390f35b6100f661026e565b6040516101039190610349565b60405180910390f35b6101266004803603810190610121919061030f565b610273565b6040516101339190610349565b60405180910390f35b5f6002548261014b919061038f565b9050919050565b5f600160025f828254610165919061038f565b925050819055506002547f437ed1f411e2ba750c55a12da4f261a1b9f2db8c9046de9ab6119538e1def34560405160405180910390a2600254905090565b

In [211]:
vinstance = util.compile_and_deploy_contract(EXAMPLE_PATH + "Visibility/Visibility.sol", compiler="solc", w3conn=w3)

In [212]:
hex(vinstance.functions.pub_int().call())

'0x20'

In [213]:
hex(vinstance.functions.pub_change().call())

'0x21'

In [214]:
hex(vinstance.functions.view_priv_int(1).call())

'0x41'

In [215]:
tx_hash = vinstance.functions.pub_change().transact({"from":w3.eth.accounts[0],})
tx_hash
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0xb35d8cf90d55a37fb22a69b494037b22fe538c0632e1fdc0267798ce505575b8'),
 'transactionIndex': 0,
 'blockNumber': 19,
 'blockHash': HexBytes('0x38fcf68c895947193143f03a0948686ce9ef641287edd017b9caeb428e294cce'),
 'from': '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 'to': '0x0E696947A06550DEf604e82C26fd9E493e576337',
 'cumulativeGasUsed': 41981,
 'gasUsed': 41981,
 'contractAddress': None,
 'logs': [AttributeDict({'address': '0x0E696947A06550DEf604e82C26fd9E493e576337',
   'blockHash': HexBytes('0x38fcf68c895947193143f03a0948686ce9ef641287edd017b9caeb428e294cce'),
   'blockNumber': 19,
   'data': HexBytes('0x'),
   'logIndex': 0,
   'removed': False,
   'topics': [HexBytes('0x437ed1f411e2ba750c55a12da4f261a1b9f2db8c9046de9ab6119538e1def345'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000041')],
   'transactionHash': HexBytes('0xb35d8cf90d55a37fb22a69b494037b22fe538c0632e1fdc0267798ce505575b8'),
   'transactionInde

In [216]:
vinstance.events.pchange().process_receipt(tx_receipt)

/smartenv/venv/lib/python3.10/site-packages/web3/contract/base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xb35d8cf90d55a37fb22a69b494037b22fe538c0632e1fdc0267798ce505575b8') and logIndex: 0 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
/smartenv/venv/lib/python3.10/site-packages/web3/contract/base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xb35d8cf90d55a37fb22a69b494037b22fe538c0632e1fdc0267798ce505575b8') and logIndex: 1 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(


(AttributeDict({'args': AttributeDict({'v': 33}),
  'event': 'pchange',
  'logIndex': 2,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xb35d8cf90d55a37fb22a69b494037b22fe538c0632e1fdc0267798ce505575b8'),
  'address': '0x0E696947A06550DEf604e82C26fd9E493e576337',
  'blockHash': HexBytes('0x38fcf68c895947193143f03a0948686ce9ef641287edd017b9caeb428e294cce'),
  'blockNumber': 19}),)

In [217]:
vinstance.events.echange().process_receipt(tx_receipt)

/smartenv/venv/lib/python3.10/site-packages/web3/contract/base_contract.py:185: UserWarning: The log with transaction hash: HexBytes('0xb35d8cf90d55a37fb22a69b494037b22fe538c0632e1fdc0267798ce505575b8') and logIndex: 2 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(


(AttributeDict({'args': AttributeDict({'v': 65}),
  'event': 'echange',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xb35d8cf90d55a37fb22a69b494037b22fe538c0632e1fdc0267798ce505575b8'),
  'address': '0x0E696947A06550DEf604e82C26fd9E493e576337',
  'blockHash': HexBytes('0x38fcf68c895947193143f03a0948686ce9ef641287edd017b9caeb428e294cce'),
  'blockNumber': 19}),)

In [218]:
vinstance.events.ichange().process_receipt(tx_receipt)

(AttributeDict({'args': AttributeDict({'v': 49}),
  'event': 'ichange',
  'logIndex': 1,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0xb35d8cf90d55a37fb22a69b494037b22fe538c0632e1fdc0267798ce505575b8'),
  'address': '0x0E696947A06550DEf604e82C26fd9E493e576337',
  'blockHash': HexBytes('0x38fcf68c895947193143f03a0948686ce9ef641287edd017b9caeb428e294cce'),
  'blockNumber': 19}),)